In [21]:
import cv2
import numpy as np
import os
import time
import mediapipe as mp


In [22]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils


In [3]:
# Convert image and make detections
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

# Draw landmarks
def draw_styled_landmarks(image, results):
    # Face
    mp_drawing.draw_landmarks(
        image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
        mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    )
    # Pose
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    )
    # Left Hand
    mp_drawing.draw_landmarks(
        image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
    )
    # Right Hand
    mp_drawing.draw_landmarks(
        image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
    )

# Extract keypoints from the landmarks
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] 
                     for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    
    face = np.array([[res.x, res.y, res.z] 
                     for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    
    lh = np.array([[res.x, res.y, res.z] 
                   for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    
    rh = np.array([[res.x, res.y, res.z] 
                   for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])


In [4]:
# Convert image and make detections
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

# Draw landmarks
def draw_styled_landmarks(image, results):
    # Face
    mp_drawing.draw_landmarks(
        image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
        mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    )
    # Pose
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    )
    # Left Hand
    mp_drawing.draw_landmarks(
        image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
    )
    # Right Hand
    mp_drawing.draw_landmarks(
        image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
    )

# Extract keypoints from the landmarks
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] 
                     for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    
    face = np.array([[res.x, res.y, res.z] 
                     for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    
    lh = np.array([[res.x, res.y, res.z] 
                   for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    
    rh = np.array([[res.x, res.y, res.z] 
                   for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    
    return np.concatenate([pose, face, lh, rh])


In [5]:
DATA_PATH = os.path.join('MP_Data')  # Folder to store data
actions = np.array(['help', 'hello', 'i need help'])  # Labels
no_sequences = 30  # Videos per action
sequence_length = 30  # Frames per video

# Create folders
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass


In [6]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):

                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)

                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    cv2.putText(image, f'Collecting frames for {action} Video {sequence}', (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.waitKey(200)
                else:
                    cv2.putText(image, f'Collecting frames for {action} Video {sequence}', (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)

                # Show image
                cv2.imshow('OpenCV Feed', image)

                # Save keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

cap.release()
cv2.destroyAllWindows()


In [7]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Constants
DATA_PATH = os.path.join('MP_Data')
actions = np.array(['help', 'hello', 'i need help'])
no_sequences = 30
sequence_length = 30

# Collect sequences, labels
label_map = {label: num for num, label in enumerate(actions)}

sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            path = os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy")
            window.append(np.load(path))
        sequences.append(window)
        labels.append(label_map[action])

X = np.array(sequences)
y = to_categorical(labels).astype(int)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)


In [12]:
print(X.shape)  # Should be (num_samples, 30, 1662)
print(y.shape)  # Should be (num_samples, 3)


(90, 30, 1662)
(90, 3)


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Step 1: Build Model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(Dropout(0.5))

model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

# Step 2: Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)

# Step 3: Early Stopping Callback
es = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Step 4: Train Model
model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=300,
    callbacks=[es]
)


Epoch 1/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 297ms/step - categorical_accuracy: 0.3817 - loss: 1.0936 - val_categorical_accuracy: 0.4000 - val_loss: 1.0777
Epoch 2/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - categorical_accuracy: 0.7965 - loss: 1.0328 - val_categorical_accuracy: 0.4000 - val_loss: 1.0626
Epoch 3/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - categorical_accuracy: 0.8552 - loss: 0.9920 - val_categorical_accuracy: 0.4000 - val_loss: 1.0593
Epoch 4/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - categorical_accuracy: 0.8787 - loss: 0.9362 - val_categorical_accuracy: 0.4000 - val_loss: 1.0462
Epoch 5/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - categorical_accuracy: 0.8357 - loss: 0.9111 - val_categorical_accuracy: 0.8000 - val_loss: 1.0296
Epoch 6/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - categorical_accuracy: 0.9002 - loss: 0.8322 - val_categorical_accuracy: 0.4000 - val_loss: 0.9993
Epoch 7/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - categorical_accuracy: 0.9100 - loss: 0.7556 - v

In [16]:
model.save('action_model.h5')


In [11]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))
model.save('action.h5')  # Save the model


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - categorical_accuracy: 0.3152 - loss: 1.0922 - val_categorical_accuracy: 0.0000e+00 - val_loss: 1.1077
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - categorical_accuracy: 0.3542 - loss: 1.0889 - val_categorical_accuracy: 0.0000e+00 - val_loss: 1.1106
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - categorical_accuracy: 0.3269 - loss: 1.0908 - val_categorical_accuracy: 0.0000e+00 - val_loss: 1.1095
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - categorical_accuracy: 0.3425 - loss: 1.0874 - val_categorical_accuracy: 0.0000e+00 - val_loss: 1.1074
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - categorical_accuracy: 0.3152 - loss: 1.0890 - val_categorical_accuracy: 0.0000e+00 - val_loss: 1.1033
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - categorical_accuracy: 0.3699 - loss: 1.0741 - val_categorical_accuracy: 0.0000e+00 - val_loss: 1.1016
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - categorical_accuracy: 0

In [ ]:
c

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
Prediction: help, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Prediction: help, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction: help, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Prediction: help, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Prediction: help, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Prediction: help, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction: help, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Prediction: help, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction: help, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction: help, Confidence: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction: help, Confidence: 0.99
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction: help, Confidence: 0.97
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Prediction: help, Confidence: 0.94
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: 

In [20]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import geocoder
import webbrowser
from playsound import playsound
from twilio.rest import Client
import threading

# Load the trained model
model = load_model('action_model.h5')
actions = np.array(['help', 'hello', 'i need help'])

# Twilio credentials
account_sid = "ACb345ac836525341898bba876d19dff6b"
auth_token = "b60914f7bba87864b7bf6f9dc63b4707"
twilio_from = "+14155238886"  # Default sandbox number (or your Twilio number)
twilio_to = "+918707275199"

# MediaPipe setup
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] 
                     for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] 
                     for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] 
                   for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] 
                   for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

def get_geolocation():
    g = geocoder.ip('me')
    return g.latlng

def emergency_alert():
    latlng = get_geolocation()
    print(f"[ALERT] 'i need help' detected. Location: {latlng}")

    # Open location in browser
    if latlng:
        webbrowser.open(f"https://www.google.com/maps?q={latlng[0]},{latlng[1]}")

    # Play alert sound
    try:
        playsound("alert.mp3")
    except Exception as e:
        print(f"[Sound Alert Error]: {e}")

    # Send SMS via Twilio
    try:
        client = Client(account_sid, auth_token)
        message = client.messages.create(
            body=f'🚨 Emergency Alert: "I need help" detected!\nLocation: https://www.google.com/maps?q={latlng[0]},{latlng[1]}',
            from_=twilio_from,
            to=twilio_to
        )
        print(f"[SMS Sent] SID: {message.sid}")
    except Exception as e:
        print(f"[SMS Failed]: {e}")

# Real-time prediction loop
sequence = []
sentence = []
threshold = 0.8
alert_triggered = False

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,
                          min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, holistic)
        draw_styled_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predicted_action = actions[np.argmax(res)]

            if res[np.argmax(res)] > threshold:
                if len(sentence) == 0 or predicted_action != sentence[-1]:
                    sentence.append(predicted_action)

                    if predicted_action == 'i need help' and not alert_triggered:
                        alert_triggered = True
                        threading.Thread(target=emergency_alert).start()

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Display result
            cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 531ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━